In [4]:
from crossref.restful import Works
import json

In [5]:
def getPaperItem(doistr,html=True,fromjournal=True):
    if doistr.startswith("http://dx.doi.org"):
        doilink = doistr
    else:
        doilink = "http://dx.doi.org/" + doistr
    wk = Works().doi(doistr)
    # Authors:
    authors = []
    for aa in wk['author']:
        authors.append(f"{aa['given']} {aa['family']}")
    if len(aa) == 1:
        pstring = authors[0]
    else:
        pstring = ", ".join(authors[:-1]) + ", and " + authors[-1]
    # Title:
    if html:
        pstring += f", <a href='{doilink}'>\"{wk['title'][0]}\"</a>"        
    else:
        pstring += f", \"{wk['title'][0]}\""
    # Journal and Date:
    if 'published-print' in wk.keys():
        pubyear = wk['published-print']['date-parts'][0][0]        
        flagpublished = True
    else:
        pubyear = wk['created']['date-parts'][0][0]
        flagpublished = False    
    datestr = f"{pubyear}"
    if fromjournal:
        pstring += f", {wk['container-title'][0]}, {datestr}." if flagpublished else \
                f", accepted for publication in {wk['container-title'][0]}, {datestr}."
    else: 
        pstring +=  f", in {wk['event']['name']}, {datestr}."
    pstring += f" DOI: {doistr}"
    return pubyear,pstring


def getManPaperItem(paperdata,html=True):
    pstring = paperdata['authors']  
    plink = paperdata['link']
    ptitle = paperdata['title']  
    if html and (plink != ""):
        pstring += f", <a href='{plink}'>\"{ptitle}\"</a>"        
    else:
        pstring += f", \"{ptitle}\""
    # Journal and Date:
    pubyear = paperdata['year']      
    datestr = f"{pubyear}"
    pstring +=  f", in {paperdata['conference']}, {datestr}."
    return pubyear,pstring,paperdata['lang']

In [6]:
doilist = []
with open("journalpaperlist.txt","r") as ff:
    doilist = ff.readlines()

mylist = []
for dd in doilist:
    if len(dd) > 3:
        mylist.append(getPaperItem(dd))

mylist = sorted(mylist,key=lambda x: x[0],reverse=True)

respstr = "<ol>\n"
for dd in mylist:
    respstr += "<li>"
    respstr += dd[1]
    respstr += "</li>\n"
respstr += "</ol>\n"

with open("src/journalpubs.html","w+") as ff:
    ff.write(respstr)

In [106]:
mylist = []

# Papers with DOI:
with open("confpaperswithdoi.txt","r") as ff:
    for ll in ff.readlines():
        if len(ll) > 5:
            aux = ll.split("\t")
            mylist.append( getPaperItem(aux[1],fromjournal=False) + (aux[0],) )

# Papers without DOI:
dtwdoi = json.load(open('confpaperswoutdoi.json'))
for kk in dtwdoi['paperlist'].keys():
    mylist.append(getManPaperItem(dtwdoi['paperlist'][kk]))   

mylist = sorted(mylist,key=lambda x: x[0],reverse=True)

respstr = "<ol>\n"
for dd in mylist:
    respstr += "<li><span class='tag is-success is-light mr-2'>In portuguese</span>" if dd[2] == "pt" else "<li>"
    respstr += dd[1]
    respstr += "</li>\n"
respstr += "</ol>\n"

with open("src/conferencepubs.html","w+") as ff:
    ff.write(respstr)